In [1]:
import requests, json
import xml.etree.ElementTree as ET
import pandas as pd
import subprocess
import sys
import urllib3
urllib3.disable_warnings()

authorize_url = "https://api.login.yahoo.com/oauth2/request_auth"
token_url = "https://api.login.yahoo.com/oauth2/get_token"

#callback url specified when the application was defined
callback_uri = "https://localhost"

test_api_url = "https://fantasysports.yahooapis.com/fantasy/v2/game/nfl"

#client (application) credentials
client_id = ''
client_secret = ''

#step A - simulate a request from a browser on the authorize_url - will return an authorization code after the user is
# prompted for credentials.

authorization_redirect_url = authorize_url + '?response_type=code&client_id=' + client_id + '&redirect_uri=' + callback_uri + '&scope=openid'


print ("go to the following url on the browser and enter the code from the returned url: ")
print ("---  " + authorization_redirect_url + "  ---")
authorization_code = input('code: ')

# step I, J - turn the authorization code into a access token, etc
data = {'grant_type': 'authorization_code', 'code': authorization_code, 'redirect_uri': callback_uri}
print ("requesting access token")
access_token_response = requests.post(token_url, data=data, verify=False, allow_redirects=False, auth=(client_id, client_secret))

#print ("response")
#print (access_token_response.headers)
#print ('body: ' + access_token_response.text)

# we can now use the access_token as much as we want to access protected resources.
tokens = json.loads(access_token_response.text)
access_token = tokens['access_token']
#print ("access token: " + access_token)

api_call_headers = {'Authorization': 'Bearer ' + access_token}
api_call_response = requests.get(test_api_url, headers=api_call_headers, verify=False)

#print (api_call_response.text)

go to the following url on the browser and enter the code from the returned url: 
---  https://api.login.yahoo.com/oauth2/request_auth?response_type=code&client_id=dj0yJmk9eGRSdkwyYWFKMk1GJmQ9WVdrOVYzaFlhV2hZV0hvbWNHbzlNQT09JnM9Y29uc3VtZXJzZWNyZXQmc3Y9MCZ4PTc1&redirect_uri=https://localhost&scope=openid  ---
code: zfsdw4u6cec3y4wjjgv7v3cr7vuyqvgw
requesting access token


In [2]:
#constants
ns = '{http://fantasysports.yahooapis.com/fantasy/v2/base.rng}'
json = '?format=json'

In [3]:
def GetStandingsXML():
    team_name = []
    team_wins = []
    team_losses = []
    team_pct = []
    team_rank = []
    team_draft_grade = []
    team_faab = []
    team_moves = []
    team_trades = []
    team_PF = []
    team_PA = []
    team_keys = []

    team_name.clear()
    team_wins.clear()
    team_losses.clear()
    team_pct.clear()
    team_rank.clear()
    team_draft_grade.clear()
    team_faab.clear()
    team_moves.clear()
    team_trades.clear()
    team_PF.clear()
    team_PA.clear()
    team_keys.clear()

    standings_URL = 'https://fantasysports.yahooapis.com/fantasy/v2/league/406.l.562818/standings'
    st_r = requests.get(standings_URL, headers=api_call_headers, verify=False)
    root_st = ET.fromstring(st_r.content)
    for child in root_st.iter(ns + 'name'):
        team_name.append(child.text)
    for child in root_st.iter(ns + 'wins'):
        team_wins.append(child.text)
    for child in root_st.iter(ns + 'losses'):
        team_losses.append(child.text)
    for child in root_st.iter(ns + 'percentage'):
        team_pct.append(child.text)
    for child in root_st.iter(ns + 'points_for'):
        team_PF.append(child.text)
    for child in root_st.iter(ns + 'points_against'):
        team_PA.append(child.text)
    for child in root_st.iter(ns + 'faab_balance'):
        team_faab.append(child.text)
    for child in root_st.iter(ns + 'number_of_moves'):
        team_moves.append(child.text)
    for child in root_st.iter(ns + 'number_of_trades'):
        team_trades.append(child.text)
    for child in root_st.iter(ns + 'rank'):
        team_rank.append(child.text)
    for child in root_st.iter(ns + 'draft_grade'):
        team_draft_grade.append(child.text)
    for child in root_st.iter(ns + 'team_key'):
        team_keys.append(child.text)

    team_name.remove('Burlington and Beyond')
    standings = pd.DataFrame({
    'team_name':team_name,
    'wins':team_wins,
    'losses':team_losses,
    'pct':team_pct,
    'draft_grade':team_draft_grade,
    'PF':team_PF,
    'PA':team_PA,
    'FAAB':team_faab,
    'Moves':team_moves,
    'Trades':team_trades,
    'Key':team_keys
    })

    return team_keys, standings

In [4]:
def GetTotalvsProjected():
    week_total_points = []
    week_projected_points = []
    week_name = []

    week_total_points.clear()
    week_projected_points.clear()
    week_name.clear()

    for i in range(1,15): 
        URL = 'https://fantasysports.yahooapis.com/fantasy/v2/team/406.l.562818.t.1/stats;type=week;week=' + str(i) + json
        r = requests.get(URL, headers=api_call_headers, verify=False)
        results = r.json()
        week_total_points.append(results['fantasy_content']['team'][1]['team_points']['total'])
        week_projected_points.append(results['fantasy_content']['team'][1]['team_projected_points']['total'])
        week_name.append(results['fantasy_content']['team'][1]['team_points']['week'])

    week_stats = pd.DataFrame({
        'week':week_name,
        'total_points':week_total_points,
        'projected_points':week_projected_points
    })

    week_stats['total_points'] = week_stats['total_points'].astype(float)
    week_stats['projected_points'] = week_stats['projected_points'].astype(float)
    week_stats['difference'] = (week_stats['total_points'] - week_stats['projected_points'])

    return week_stats

In [5]:
def GetRosterStats(team_keys):
    player_manager = []
    player = []
    position = []
    primary_pos = []
    p_total = []

    player_manager.clear()
    player.clear()
    position.clear()
    primary_pos.clear()
    p_total.clear()

    week = 14

    for i in range(week,week+1):
        for key in team_keys:
            URL = 'https://fantasysports.yahooapis.com/fantasy/v2/team/' + str(key) + '/roster/players/stats;type=week;week=' + str(i) + json
            #print(URL)
            r = requests.get(URL, headers=api_call_headers, verify=False)
            #print(r.text)
            results = r.json()
            count = results['fantasy_content']['team'][1]['roster']['0']['players']['count']
            #print(count)
            for j in range(0,count):
                player_manager.append(results['fantasy_content']['team'][0][2]['name'])
                p = results['fantasy_content']['team'][1]['roster']['0']['players'][str(j)]['player']
                player.append(p[0][2]['name']['full'])
                position.append(p[1]['selected_position'][1]['position'])
                if str(key) == '406.l.562818.t.1':
                    p_total.append(p[3]['player_points']['total'])
                else:
                    p_total.append(p[2]['player_points']['total'])
                #print(j)
                if 'injury' in str(p[0][4]):
                    primary_pos.append(p[0][15]['primary_position'])
                elif 'status' in str(p[0][3]) and 'injury' not in str(p[0][4]):
                    primary_pos.append(p[0][14]['primary_position'])
                else:
                    primary_pos.append(p[0][13]['primary_position'])

    #print(len(player_manager))
    #print(len(player))
    #print(len(position))
    #print(len(primary_pos))
    #print(len(p_total))

    player_stats = pd.DataFrame({
        'manager':player_manager,
        'player':player,
        'position':position,
        'primary_pos':primary_pos,
        'total':p_total
    })

    player_stats['total'] = player_stats['total'].astype(float)
    #player_stats.nlargest(10,'total')

    #bench_players = player_stats.loc[player_stats['position'].str.contains('BN')]
    #bench_players.loc[bench_players['manager'].str.contains('This')]
    #bench_players.groupby('manager').sum()
    return player_stats

In [6]:
def GetBounties():
    week = []
    recap = []
    winner_team = []
    winner_total = []
    loser_team = []
    loser_total = []
    overall_week = []
    overall_team = []
    overall_total = []

    week.clear()
    recap.clear()
    winner_team.clear()
    loser_team.clear()
    winner_total.clear()
    loser_total.clear()
    overall_week.clear()
    overall_team.clear()
    overall_total.clear()

    for i in range(1,15): 
        URL = 'https://fantasysports.yahooapis.com/fantasy/v2/league/406.l.562818/scoreboard;type=week;week=' + str(i) + json
        r = requests.get(URL, headers=api_call_headers, verify=False)
        results = r.json()
        count = results['fantasy_content']['league'][1]['scoreboard']['0']['matchups']['count']
        for j in range(0,count):
            week.append(results['fantasy_content']['league'][1]['scoreboard']['week'])
            m = results['fantasy_content']['league'][1]['scoreboard']['0']['matchups'][str(j)]['matchup']
            recap.append(m['matchup_recap_title'])
            winner_team_key = m['winner_team_key']
            if winner_team_key == str(m['0']['teams']['0']['team'][0][0]['team_key']):   
                winner_team.append(m['0']['teams']['0']['team'][0][2]['name'])
                winner_total.append(m['0']['teams']['0']['team'][1]['team_points']['total'])
            else:
                loser_team.append(m['0']['teams']['0']['team'][0][2]['name'])
                loser_total.append(m['0']['teams']['0']['team'][1]['team_points']['total'])
            if winner_team_key == str(m['0']['teams']['1']['team'][0][0]['team_key']):   
                winner_team.append(m['0']['teams']['1']['team'][0][2]['name'])
                winner_total.append(m['0']['teams']['1']['team'][1]['team_points']['total'])
            else:
                loser_team.append(m['0']['teams']['1']['team'][0][2]['name'])
                loser_total.append(m['0']['teams']['1']['team'][1]['team_points']['total'])
            overall_team.append(m['0']['teams']['0']['team'][0][2]['name'])
            overall_total.append(m['0']['teams']['0']['team'][1]['team_points']['total'])
            overall_team.append(m['0']['teams']['1']['team'][0][2]['name'])
            overall_total.append(m['0']['teams']['1']['team'][1]['team_points']['total'])
            overall_week.append(results['fantasy_content']['league'][1]['scoreboard']['week'])
            overall_week.append(results['fantasy_content']['league'][1]['scoreboard']['week'])

    matchups = pd.DataFrame({
        'week':week,
        'winner_team':winner_team,
        'winner_total':winner_total,
        'loser_total':loser_total,
        'loser_team':loser_team,
        'recap':recap
    })

    matchups['winner_total'] = matchups['winner_total'].astype(float)
    matchups['loser_total'] = matchups['loser_total'].astype(float)

    matchups.insert(3,"diff",(matchups['winner_total']-matchups['loser_total']))
    #matchups['diff'] = matchups['diff'].abs()
    #matchups.sort_values('diff')
    #matchups.nsmallest(10,'diff')
    #matchups.nlargest(5,'loser_total')
    
    overall_stats = pd.DataFrame({
    'week':overall_week,
    'team':overall_team,
    'total':overall_total
    })

    overall_stats['total'] = overall_stats['total'].astype(float)
    overall_stats['week'] = overall_stats['week'].astype(int) 

    #overall_stats = overall_stats.nlargest(100,'total')
    #overall_stats.groupby('week').head(1).sort_values(by=['week'])
    return matchups, overall_stats

In [7]:
def GetPlayerWeekStats():
    player_week = []
    player_names = []
    player_totals = []

    player_week.clear()
    player_names.clear()
    player_totals.clear()

    week = 14

    for i in range(0,1150,25):
        #print(i)
        for j in range(week, week+1):
            URL3 = 'https://fantasysports.yahooapis.com/fantasy/v2/league/406.l.562818/players;start=' + str(i) + '/stats;type=week;week=' + str(j) + json
            try:
                r3 = requests.get(URL3, headers=api_call_headers, verify=False)
            except requests.exceptions.RequestException as e:  # This is the correct syntax
                raise SystemExit(e)
            #print(r3.text)
            #print(len(r3.text))
            results3 = r3.json()
            count = results3['fantasy_content']['league'][1]['players']['count']
            for k in range(0,count):
                p = results3['fantasy_content']['league'][1]['players'][str(k)]['player']
                player_week.append(p[1]['player_points']['0']['week'])
                player_names.append(p[0][2]['name']['full'])
                player_totals.append(p[1]['player_points']['total'])

    all_player_stats = pd.DataFrame({
        'week':player_week,
        'name':player_names,
        'total':player_totals
    })

    all_player_stats['total'] = all_player_stats['total'].astype(float)

    return all_player_stats
    #all_player_stats.nlargest(10,'total')

In [25]:
def GetInjuryReport():
    player_week = []
    player_names = []
    player_status = []
    player_injury = []

    player_week.clear()
    player_names.clear()
    player_status.clear()
    player_injury.clear()

    week = 15

    for i in range(0,1150,25):
        #print(i)
        for j in range(week, week+1):
            URL3 = 'https://fantasysports.yahooapis.com/fantasy/v2/league/406.l.562818/players;start=' + str(i) + '/stats;type=week;week=' + str(j) + json
            try:
                r3 = requests.get(URL3, headers=api_call_headers, verify=False)
            except requests.exceptions.RequestException as e:  # This is the correct syntax
                raise SystemExit(e)
            #print(r3.text)
            #print(len(r3.text))
            results3 = r3.json()
            count = results3['fantasy_content']['league'][1]['players']['count']
            for k in range(0,count):
                p = results3['fantasy_content']['league'][1]['players'][str(k)]['player']
                if 'injury' in str(p[0][4]):
                    player_week.append(p[1]['player_points']['0']['week'])
                    player_names.append(p[0][2]['name']['full'])
                    player_status.append(p[0][3]['status'])
                    player_injury.append(p[0][4]['injury_note'])
                elif 'status' in str(p[0][3]) and 'injury' not in str(p[0][4]):
                    player_week.append(p[1]['player_points']['0']['week'])
                    player_names.append(p[0][2]['name']['full'])
                    player_status.append(p[0][3]['status'])
                    player_injury.append('N/A')

    injury_report = pd.DataFrame({
        'week':player_week,
        'player':player_names,
        'status':player_status,
        'injury':player_injury
    })

    return injury_report

In [26]:
matchups, overall_stats = GetBounties()
team_keys, standings = GetStandingsXML()
injury_report = GetInjuryReport()
roster_stats = GetRosterStats(team_keys)

In [22]:
pd.set_option('display.max_colwidth', None)

#matchups['recap'].loc[matchups['week'].str.contains('14')]
matchups.loc[matchups['week'].str.contains('14')]

,week,winner_team,winner_total,diff,loser_total,loser_team,recap
91,14,Buckle Up Bi*ch,150.35,0.58,149.77,Wisest Wizards,Back-and-forth battle ends with Buckle Up Bi*ch coming back to knock off Wisest Wizards in 150.35-149.77 struggle
92,14,LabraDood,149.32,22.60,126.72,The Pickle Party,LabraDood come back to knock off The Pickle Party in 149.32-126.72 drubbing
93,14,Aaron Rodger’s Horse Dewormers,149.04,28.01,121.03,Rub my Chubb,Aaron Rodger’s Horse Dewormers knock Rub my Chubb from 1st place in 149.04-121.03 drubbing
94,14,Fairytale Footballrz,153.69,20.44,133.25,The JV Squad,"Fairytale Footballrz brush off projections, defeat The JV Squad in 153.69-133.25 beatdown"
95,14,Adams' Apple,142.75,56.46,86.29,Born Tua Be Wild,Adams' Apple hand Born Tua Be Wild fourth consecutive loss in 142.75-86.29 landslide
96,14,Hero Squad,136.22,27.68,108.54,Ginger Snaps,"Hero Squad climb into top spot, 136.22-108.54"
97,14,Lou So Phancy Paul,148.57,12.19,136.38,This Is Fine,"Lou So Phancy Paul knock off This Is Fine for fourth straight win, 148.57-136.38"


In [23]:
matchups.nlargest(10,'loser_total')

,week,winner_team,winner_total,diff,loser_total,loser_team,recap
6,1,Lou So Phancy Paul,188.96,10.72,178.24,This Is Fine,"Lou So Phancy Paul defeat This Is Fine, 188.96-178.24"
37,6,This Is Fine,169.24,10.62,158.62,Fairytale Footballrz,"This Is Fine come back to hand Fairytale Footballrz third straight loss, 169.24-158.62"
31,5,Born Tua Be Wild,184.73,31.58,153.15,Fairytale Footballrz,Born Tua Be Wild defeat Fairytale Footballrz for fourth straight win in 184.73-153.15 rout
91,14,Buckle Up Bi*ch,150.35,0.58,149.77,Wisest Wizards,Back-and-forth battle ends with Buckle Up Bi*ch coming back to knock off Wisest Wizards in 150.35-149.77 struggle
34,5,Rub my Chubb,181.78,32.56,149.22,Buckle Up Bi*ch,Rub my Chubb come back to knock off Buckle Up Bi*ch in 181.78-149.22 landslide
25,4,Lou So Phancy Paul,155.11,8.83,146.28,Buckle Up Bi*ch,"Lou So Phancy Paul overcome multiple busts, knock off Buckle Up Bi*ch, 155.11-146.28"
19,3,Buckle Up Bi*ch,147.64,1.52,146.12,LabraDood,Buckle Up Bi*ch come back to hand LabraDood third straight loss in 147.64-146.12 squeaker
73,11,Fairytale Footballrz,168.14,22.70,145.44,Rub my Chubb,Back-and-forth battle ends with Fairytale Footballrz knocking off Rub my Chubb in 168.14-145.44 drubbing
52,8,Hero Squad,160.16,14.76,145.40,The JV Squad,"Hero Squad come back to top The JV Squad for sixth straight win, 160.16-145.40"
42,7,Wisest Wizards,146.89,2.66,144.23,Rub my Chubb,"Despite top scorer Kupp, Rub my Chubb fall to Wisest Wizards in 146.39-144.23 struggle"


In [46]:
matchups.nsmallest(10,'diff')

,week,winner_team,winner_total,diff,loser_total,loser_team,recap
91,14,Buckle Up Bi*ch,150.35,0.58,149.77,Wisest Wizards,Back-and-forth battle ends with Buckle Up Bi*ch coming back to knock off Wisest Wizards in 150.35-149.77 struggle
16,3,Hero Squad,137.74,0.75,136.99,Aaron Rodger’s Horse Dewormers,"Despite top scorer Williams, The Antonio Clowns fall to Hero Squad in 137.74-136.99 nailbiter"
80,12,The JV Squad,136.83,1.01,135.82,Adams' Apple,The JV Squad come back to knock off Adams' Apple in 136.83-135.82 nailbiter
64,10,The Pickle Party,141.74,1.23,140.51,Buckle Up Bi*ch,The Pickle Party knock off Buckle Up Bi*ch in 141.74-140.51 cliff-hanger
33,5,Adams' Apple,129.72,1.39,128.33,Ginger Snaps,Adams' Apple knock off Ginger Snaps to remain perfect in 129.72-128.33 squeaker
19,3,Buckle Up Bi*ch,147.64,1.52,146.12,LabraDood,Buckle Up Bi*ch come back to hand LabraDood third straight loss in 147.64-146.12 squeaker
85,13,The Pickle Party,122.76,1.79,120.97,Born Tua Be Wild,The Pickle Party defeat Born Tua Be Wild for fourth straight win in 122.76-120.97 squeaker
26,4,Hero Squad,116.62,2.32,114.30,This Is Fine,Hero Squad hand This Is Fine fourth straight loss in 116.62-114.20 thriller
42,7,Wisest Wizards,146.89,2.66,144.23,Rub my Chubb,"Despite top scorer Kupp, Rub my Chubb fall to Wisest Wizards in 146.39-144.23 struggle"
27,4,Adams' Apple,111.29,2.80,108.49,Rub my Chubb,Offensively challenged Adams' Apple come back to beat Rub my Chubb in 111.29-108.49 squeaker


In [35]:
injury_report.loc[injury_report['status'].str.contains('COVID')].count()

week      29
player    29
status    29
injury    29
dtype: int64

In [30]:
roster_injuries = pd.merge(injury_report, roster_stats, on=['player'])
roster_injuries
roster_injuries.loc[roster_injuries['status'].str.contains('COVID')]

,week,player,status,injury,manager,position,primary_pos,total
5,15,Mark Ingram II,COVID-19,COVID-19,Wisest Wizards,BN,RB,0.0
12,15,Odell Beckham Jr.,COVID-19,COVID-19,LabraDood,WR,WR,19.7
13,15,Jarvis Landry,COVID-19,COVID-19,Fairytale Footballrz,BN,WR,15.1
23,15,Christian McCaffrey,COVID-19,Ankle,The Pickle Party,IR,RB,0.0
26,15,Jamaal Williams,COVID-19,COVID-19,Buckle Up Bi*ch,IR,RB,0.0
39,15,Darrell Henderson Jr.,COVID-19,COVID-19,Hero Squad,IR,RB,0.0
42,15,Alexander Mattison,COVID-19,COVID-19,Ginger Snaps,RB,RB,2.7
45,15,Myles Gaskin,COVID-19,COVID-19,The Pickle Party,BN,RB,0.0


In [45]:
overall_stats[overall_stats['week'] == 14].nlargest(14,'total')

,week,team,total
188,14,Fairytale Footballrz,153.69
183,14,Buckle Up Bi*ch,150.35
182,14,Wisest Wizards,149.77
185,14,LabraDood,149.32
186,14,Aaron Rodger’s Horse Dewormers,149.04
194,14,Lou So Phancy Paul,148.57
191,14,Adams' Apple,142.75
195,14,This Is Fine,136.38
193,14,Hero Squad,136.22
189,14,The JV Squad,133.25
